In [21]:
import numpy as np
import pandas as pd
import nltk
import re
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedLineDocument
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split


In [29]:
testing = True # Set to True to use small files for quick testing, false to use the entire training data
if (testing):
    filename = 'Data/mergedDataSet_test.csv' #Sample file used for quick testing
    commentsFileName = 'Data/comments_small.csv'
else:
    filename = 'Data/mergedDataSet.csv' #The complete file
    commentsFileName = 'Data/comments.csv'

df = pd.read_csv(filename)

In [31]:
# nltk.download('stopwords') # Download it if not already downloaded
stop = stopwords.words('english')
def RemoveStopWords(row):
    row = row.lower().split() # converting to lower case and splitting
    str1 = ''
    for item in row:
        if item not in stop: #removing stop words
            item = re.sub(r'[^\w\s]','',item) #removing punctutions
            str1 += (item + ' ')
    return str1
target = df['merged_rating'].values #Saving the target variable as a numpy array
df = df['comment_text'].apply(RemoveStopWords)


In [32]:
df.to_csv(commentsFileName,index=False) # Writing the comments to a CSV file to be read by TaggedLineDocument next
documents = TaggedLineDocument(commentsFileName) # Tags each sentence (0,1,2,...)

In [33]:
modelSize = 500 # Will represent each comment with a vector of size 500
modelWindow = 8 # Will look at 8 words together to do its calculations
model = Doc2Vec(documents, modelSize, modelWindow, min_count=5, workers=4)
# model.save('model_s500_w8') # Uncomment this to save the model to disk

In [34]:
#Creating a numpy array to keep the data and to be used by a machine learning model as the feature vector
data = np.zeros(model.docvecs[0].shape)
for i in range(len(model.docvecs)):
    data = np.vstack((data,model.docvecs[i]))
data = np.delete(data,0,0)

In [11]:
#Creating validation sets
X_train, X_val, y_train, y_val = train_test_split(data,target, test_size=0.2, random_state=0)